## Preprocessing

In [ ]:
%cd vlsp-mt

In [ ]:
!python scripts/dedup_minhash.py \
    --src data/clean/train.en \
    --tgt data/clean/train.vi \
    --out_dir data/dedup \
    --threshold 0.8 \
    --dedup_by both \
    --rep_strategy longest \
    --smart_threshold

## Train

In [ ]:
!pip install evaluate

In [ ]:
!python scripts/train_qwen_lora.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --direction en2vi \
    --src data/clean/train.en \
    --tgt data/clean/train.vi \
    --val_src data/clean/dev.en \
    --val_tgt data/clean/dev.vi \
    --run_id en2vi_v1 \
    --epochs 2 \
    --batch_size 8 \
    --grad_accum 4 \
    --lr 1.5e-4 \
    --lora_r 32 \
    --lora_alpha 64 \
    --neftune_alpha 3.0 \
    --bleu_sample_size 200 \
    --eval_steps 1000 \
    --early_stopping_patience 2 \
    --no_grad_checkpoint  \
    --max_len 256

In [ ]:
# 1. Tạo thư mục đích trước
!mkdir -p data/rl_subset

# 2. Chạy lệnh cắt file
!head -n 20000 data/dedup/train.en > data/rl_subset/en.txt
!head -n 20000 data/dedup/train.vi > data/rl_subset/vi.txt

!ls -lh data/rl_subset/

In [ ]:
!python scripts/rl_train_grpo.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --sft_adapter runs/en2vi_v1/lora_en2vi_sft \
    --init_adapter runs/en2vi_v1/lora_en2vi_sft \
    --rl_src data/rl_subset/en.txt \
    --rl_tgt data/rl_subset/vi.txt \
    --run_id en2vi_v1_rl \
    --direction en2vi \
    --epochs 1 \
    --batch_size 4 \
    --grad_accum_steps 8 \
    --lr 3e-6 \
    --kl_coef 0.02 \
    --temperature 0.8 \
    --max_new_tokens 64

# augemnet

In [ ]:
!python scripts/back_translate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/vi2en_v1/lora_vi2en_sft \
    --input data/rl_subset/vi.txt \
    --output data/augment/bt.en \
    --direction vi2en \
    --batch_size 16 \
    --temperature 0.7

In [ ]:
!cat data/clean/test.en data/augment/bt.en > data/augment/train_aug_en2vi.en
!cat data/clean/test.vi data/rl_subset/vi.txt > data/augment/train_aug_en2vi.vi

In [ ]:
!python scripts/train_qwen_lora.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --direction en2vi \
    --src data/augment/train_aug.en \
    --tgt data/augment/train_aug.vi \
    --val_src data/clean/dev.en \
    --val_tgt data/clean/dev.vi \
    --run_id en2vi_v2_aug \
    --epochs 2 \
    --batch_size 8 \
    --grad_accum 4 \
    --lr 1.5e-4 \
    --lora_r 32 \
    --lora_alpha 64 \
    --neftune_alpha 3.0 \
    --bleu_sample_size 200 \
    --eval_steps 500 \
    --early_stopping_patience 3 \
    --no_grad_checkpoint  \
    --max_len 256

## Eval

In [1]:
# Unseen Test

In [52]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/en2vi_v1/lora_en2vi_sft \
    --direction en2vi \
    --input data/raw/test_unseen_v3.en.txt \
    --output outputs/test_unseen_v3.hyp.vi \
    --batch_size 24 \
    --num_beams 4 \
    --repetition_penalty 1.1

2025-12-22 20:57:03.509803: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 20:57:03.529379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766437023.553848  324096 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766437023.560770  324096 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766437023.578067  324096 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [53]:
!python scripts/eval_bleu.py \
    --hyp outputs/test_unseen_v3.hyp.vi \
    --ref data/raw/test_unseen_v3.vi.txt \
    --src data/raw/test_unseen_v3.en.txt \
    --show_worst 10

Loading hypothesis: outputs/test_unseen_v3.hyp.vi
Loading reference: data/raw/test_unseen_v3.vi.txt
Loading source: data/raw/test_unseen_v3.en.txt

Evaluating 1152 sentence pairs...

EVALUATION RESULTS
  BLEU:    51.11
  chrF++:  64.96
  chrF:    65.13
  TER:     42.43 (lower is better)
  METEOR:  73.89

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 504 | BLEU: 3.9 | chrF: 25.7
  SRC: Initial experiences of Halo-gravity traction in treatment of extremely severe adult idiopathic scoliosis
  HYP: Kinh nghiệm bước đầu điều trị vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên người lớn vẹo cột sống không rõ căn nguyên ng

In [2]:
## Test

In [41]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/en2vi_v1/lora_en2vi_sft \
    --direction en2vi \
    --input data/clean/test.en \
    --output outputs/test_en2vi.hyp.vi \
    --batch_size 24 \
    --num_beams 4 \
    --repetition_penalty 1.1

2025-12-22 19:41:58.136004: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 19:41:58.154433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766432518.176773  304989 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766432518.183392  304989 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766432518.200299  304989 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [42]:
!python scripts/eval_bleu.py \
    --hyp outputs/test_en2vi.hyp.vi \
    --ref data/clean/test.vi \
    --src data/clean/test.en \
    --show_worst 10

Loading hypothesis: outputs/test_en2vi.hyp.vi
Loading reference: data/clean/test.vi
Loading source: data/clean/test.en

Evaluating 1000 sentence pairs...

EVALUATION RESULTS
  BLEU:    52.53
  chrF++:  66.28
  chrF:    66.36
  TER:     39.75 (lower is better)
  METEOR:  75.40

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 860 | BLEU: 1.7 | chrF: 1.4
  SRC: RESULT OF TREATMENT OF SEVERE PNEUMONIA IN CHILDREN FROM 1 MONTH TO FIVE YEARS OLD IN QUANG NINH PROVINC
  HYP: ĐÁNH GIÁ KẾT QUẢ ĐIỀU TRỊ VIÊM PHỔI NẶNG Ở TRẺ EM TỪ 1 THÁNG ĐẾN 5 TUỔI TẠI TỈNH QUẢNG NINH
  REF: Kết quả điều trị viêm phổi nặng ở trẻ từ 1tháng đến 5 tuổi tại Quảng Ninh

[2] Line 315 | BLEU: 2.8 | chrF: 5.2
  SRC: EVALUATION OF PRELIMINARY RESULTS OF ENDOSCOPIC LITHOTRIPSY THROUGH T-TUBE TRACT AT CAN THO CENTRAL GENERAL HOSPITAL FROM 2020 TO 2021
  HYP: Đánh giá kết quả bước đầu điều trị sỏi đường mật bằng nội soi tán sỏi qua đường hầm Kehr tại Bệnh viện Đa khoa Trung ương

RL

In [45]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/en2vi_v1_rl/final_model/policy \
    --direction en2vi \
    --input data/clean/test.en \
    --output outputs/test_en2vi_rl.hyp.vi \
    --batch_size 16 \
    --num_beams 4 \
    --repetition_penalty 1.1


2025-12-22 19:56:49.954842: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 19:56:49.973749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766433409.996848  308879 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766433410.003524  308879 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766433410.020875  308879 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [46]:
!python scripts/eval_bleu.py \
    --hyp outputs/test_en2vi_rl.hyp.vi \
    --ref data/clean/test.vi \
    --src data/clean/test.en \
    --show_worst 10

Loading hypothesis: outputs/test_en2vi_rl.hyp.vi
Loading reference: data/clean/test.vi
Loading source: data/clean/test.en

Evaluating 1000 sentence pairs...

EVALUATION RESULTS
  BLEU:    52.00
  chrF++:  65.88
  chrF:    65.99
  TER:     40.13 (lower is better)
  METEOR:  74.93

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 25 | BLEU: 0.6 | chrF: 10.4
  SRC: U S. clinical trials investigating the effectiveness and safety of the procedure currently are being conducted. NHỮNG KỸ THUẬT GHÉP BÊN TRONG VÀ BÊN NGOÀI GIÁC MẠC Corneal inlays and onlays are inserts placed just beneath your eye's clear surface (cornea).
  HYP: NHỮNG KỸ THUẬT GHÉP BÊN TRONG VÀ BÊN NGOÀI GIÁC MẠC Corneal inlays and onlays are inserts placed just beneath your eye 's clear surface (cornea).
  REF: Các thủ nghiệm lâm sàng của Hoa Kỳ đang điều tra hiệu quả và sự an toàn của tiên trình hiện giờ đang được tiến hành theo hướng dẫn (conduct) Corneal Inlays & Onlays Inlays g

# Public test

In [47]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/en2vi_v1/lora_en2vi_sft \
    --direction en2vi \
    --input data/raw/public_test.en.txt \
    --output outputs/public_test_en2vi_sft.hyp.vi \
    --batch_size 20 \
    --num_beams 5 \
    --repetition_penalty 1.1

2025-12-22 20:10:52.566897: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 20:10:52.586131: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766434252.609354  312460 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766434252.616040  312460 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766434252.633280  312460 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [48]:
!python scripts/eval_bleu.py \
    --hyp outputs/test_en2vi_rl.hyp.vi \
    --ref data/clean/test.vi \
    --src data/clean/test.en \
    --show_worst 10

Loading hypothesis: outputs/public_test_en2vi_sft.hyp.vi
Loading reference: data/raw/public_test.vi.txt
Loading source: data/raw/public_test.en.txt

Evaluating 3000 sentence pairs...

EVALUATION RESULTS
  BLEU:    54.19
  chrF++:  67.29
  chrF:    67.40
  TER:     39.22 (lower is better)
  METEOR:  75.85

10 WORST TRANSLATIONS (by BLEU):
--------------------------------------------------

[1] Line 2039 | BLEU: 3.1 | chrF: 27.7
  SRC: In vascular imaging, this technique provides image quality that equals or surpasses that of conventional angiography.
  HYP: Trong chẩn đoán hình ảnh mạch máu, kỹ thuật này cung cấp chất lượng hình ảnh tương đương hoặc vượt trội so với chụp mạch thông thường.
  REF: Trong kỹ thuật chụp mạch máu lớn của lồng ngực (động mạch chủ, động mạch phổi...), chụp cắt lớp điện toán đa lát cắt này cho được các hình ảnh bằng hoặc rõ hơn chụp mạch máu quy ước.

[2] Line 1898 | BLEU: 3.4 | chrF: 22.8
  SRC: This study aims to improve the understanding of pathogenesis of p